# Multimodal RAG app with LanceDB and LlamaIndex

## Video Processing

In [5]:
from moviepy import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

In [6]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
# Configure the API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Example usage (list models)
for m in genai.list_models():
    print(m)


/Users/devayushrout/Desktop/Multimodal-RAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1758216077.230457  514259 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representation of a text.',
      input_token_limit=1024,
      output_token_limit=1,
      supported_generation_methods=['embedText', 'countTextTokens'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/gemini-1.5-pro-latest',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Pro Latest',
      description=('Alias that points to the most recent production (non-experimental) release '
                   'of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 '
                   'million tokens.'),
      input_token_limit=2000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)
Model(name='m

In [8]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in 5 lines",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

1. AI learns patterns from data, like images or text.
2. It uses these patterns to make predictions or decisions.
3. This involves algorithms, a set of instructions.
4. When given new input, it applies what it learned.
5. The output is its best guess based on its training.


In [10]:
import os
os.getcwd()

'/Users/devayushrout/Desktop/Multimodal-RAG'

In [11]:
video_url = "https://www.youtube.com/watch?v=CtsRRUddV2s"

In [12]:
output_video_path = '/Users/devayushrout/Desktop/Multimodal-RAG/video_data'

In [14]:
output_folder = "/Users/devayushrout/Desktop/Multimodal-RAG/mixed_data"
output_audio_path = "/Users/devayushrout/Desktop/Multimodal-RAG/mixed_data/audio.wav"

In [16]:
filepath = output_video_path + "/input_video.mp4"
print(filepath)

/Users/devayushrout/Desktop/Multimodal-RAG/video_data/input_video.mp4


In [ ]:
from pytube import YouTube

#function to download the youtube video from the link
def download_video (url, output_path):
    yt = YouTube(url)
    metadata = {"Author": yt.author, "Title": yt.title, "Length": yt.length, "Views": yt.views, "Description": yt.description}
    yt.streams.get_highest_resolution().download(
        output_path = output_path, filename = "input_video.mp4"
    )
    return metadata

In [ ]:
from moviepy import VideoFileClip
#convert the whole video into images
def video_to_images(video_path, output_folder):
    clip = VideoFileClip(video_path)
    clip.write_images_sequence(
        os.path.join(output_folder,"frame%04d.png"), fps = 0.2
    )

In [ ]:
#convert the whole video into audio
def video_to_audio(video_path, output_audio_path):
    clip=VideoFileClip(video_path)
    audio=clip.audio
    audio.write_audiofile(output_audio_path)

In [4]:
#convert audio to text

def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    audio = sr.AudioFile(audio_path)
    
    with audio as source:
        audio_data = recognizer.record(source)
        
        try:
            text = recognizer.recognize_whisper(audio_data)
        
        except sr.UnknownValueError:
            print("Speech Recognition could not understand audio")
    
    return text